In [1]:
import time
import json
import pickle
import pandas as pd
from classes import *
from datetime import datetime
import plotly.graph_objects as go

In [13]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [14]:
def plot_data_arr(sensor_data, mat, sensor, labels, skip=0, cut=0):
    title = f"Matrix {mat}, Sensor {sensor}"
    heaters = sensor_data[f"mat_{mat}"][sensor]
    fig = go.Figure()

    for i, data in heaters.items():
        fig.add_trace(go.Scatter(x=data["Time Since PowerOn"][skip:-cut],
                                 y=data["Filtered"][skip:-cut],
                                 mode="lines",
                                 showlegend=False,
                                 name=f"H.S. {i}"))
    
    for label in labels[f"mat_{mat}"]:
        fig.add_vrect(x0=label["start"],
                      x1=label["end"],
                      annotation_text=classes[label["label"]],
                      annotation_position="top left",
                      fillcolor=colors[label["label"]],
                      opacity=0.4,
                      line_width=0)
    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")
    fig.update_layout(height=400, width=700,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    fig.update_yaxes(type="log")
    fig.show()

In [15]:
def get_df(matrix):
    with open(f"raw_data/matrix_{matrix}/{matrix}.bmerawdata") as f:
        j = json.load(f)

    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)
    return df

In [16]:
def get_tspo_val(date_str, df):
    ts = time.mktime(datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").timetuple())
    ts = int(ts)
    values = df[df["Real time clock"] == ts]["Time Since PowerOn"].values
    return values[0]

In [17]:
df = get_df(0)

In [18]:
get_tspo_val("2024-10-24 08:10:00", df)

55646259

In [38]:
label_datetimes = {
    "mat_0": [
        {"start": "2024-10-24 08:11:00",
         "end": "2024-10-24 08:35:00",
         "cls": AIR},
        {"start": "2024-10-24 08:52:00",
         "end": "2024-10-24 09:16:00",
         "cls": CNH_0},
        {"start": "2024-10-24 09:54:00",
         "end": "2024-10-24 10:18:00",
         "cls": CNH_25},
        {"start": "2024-10-24 10:54:00",
         "end": "2024-10-24 11:18:00",
         "cls": CNH_50},
        {"start": "2024-10-24 11:54:00",
         "end": "2024-10-24 12:18:00",
         "cls": CNH_75},
        {"start": "2024-10-24 12:54:00",
         "end": "2024-10-24 13:18:00",
         "cls": CNH_90},
        {"start": "2024-10-24 13:54:00",
         "end": "2024-10-24 14:18:00",
         "cls": CNH_95},
        {"start": "2024-10-24 14:54:00",
         "end": "2024-10-24 15:18:00",
         "cls": CNH_100}
    ],
    "mat_1": [
         {"start": "2024-10-24 09:21:00",
         "end": "2024-10-24 09:45:00",
         "cls": AIR},
        {"start": "2024-10-24 09:53:00",
         "end": "2024-10-24 10:17:00",
         "cls": CNH_0},
        {"start": "2024-10-24 10:54:00",
         "end": "2024-10-24 11:18:00",
         "cls": CNH_25},
        {"start": "2024-10-24 11:54:00",
         "end": "2024-10-24 12:18:00",
         "cls": CNH_50},
        {"start": "2024-10-24 12:54:00",
         "end": "2024-10-24 13:18:00",
         "cls": CNH_75},
        {"start": "2024-10-24 13:54:00",
         "end": "2024-10-24 14:18:00",
         "cls": CNH_90},
        {"start": "2024-10-24 14:54:00",
         "end": "2024-10-24 15:18:00",
         "cls": CNH_95},
        {"start": "2024-10-24 15:54:00",
         "end": "2024-10-24 16:18:00",
         "cls": CNH_100}
    ]
}

sensor_labels = {"mat_0": [], "mat_1": []}
for matrix in range(2):
    df = get_df(matrix)
    for label_time in label_datetimes[f"mat_{matrix}"]:
        start_ts = get_tspo_val(label_time["start"], df)
        end_ts = get_tspo_val(label_time["end"], df)
        cls = label_time["cls"]
        reg_target = regression_targets[cls] if cls in regression_targets else None
        
        sensor_labels[f"mat_{matrix}"].append({
            "start": start_ts,
            "end": end_ts,
            "label": cls,
            "target": reg_target
        })

In [46]:
plot_data_arr(sensor_data, 1, 7, sensor_labels, skip=2900, cut=1)

In [47]:
with open("sensor_labels.pkl", "wb") as f:
    pickle.dump(sensor_labels, f)